In [ ]:
#since we are also interested in the variable cost, we have to refer to an additional data set
#we first structure it and afterwards merge it with the other data bits

import pandas as pd
df_cost = pd.read_excel('data/cost_hospitals.xlsx')

In [ ]:
#make sure it uploaded correctly

df_cost.head()

In [ ]:
#Es werden erst Daten ab dem Jahr 2010 benötigt
df_cost = df_cost[df_cost['JAHR'] >= 2010]

In [ ]:
#zuordnen der Kantone nach Grossregionen

kanton_to_region= {
    'GE': 'Genferseeregion', 'VD': 'Genferseeregion', 'VS': 'Genferseeregion',
    'BE': 'Espace Mittelland', 'FR': 'Espace Mittelland', 'SO': 'Espace Mittelland',
    'NE': 'Espace Mittelland', 'JU': 'Espace Mittelland',
    'BS': 'Nordwestschweiz', 'BL': 'Nordwestschweiz', 'AG': 'Nordwestschweiz',
    'ZH': 'Zürich',
    'GL': 'Ostschweiz', 'SH': 'Ostschweiz', 'AR': 'Ostschweiz', 'AI': 'Ostschweiz',
    'SG': 'Ostschweiz', 'GR': 'Ostschweiz', 'TG': 'Ostschweiz',
    'LU': 'Zentralschweiz', 'UR': 'Zentralschweiz', 'SZ': 'Zentralschweiz',
    'OW': 'Zentralschweiz', 'NW': 'Zentralschweiz', 'ZG': 'Zentralschweiz',
    'TI': 'Tessin'
}

In [ ]:
#mapping anwenden
df_cost['Grossregion']= df_cost['KT'].map(kanton_to_region)


In [ ]:
df_cost.head(20)
df_cost.query("Grossregion == 'Zürich'")


In [ ]:
#Spalten entfernen, die wir sicherlich nicht mehr benötigen

df_cost= df_cost.drop(columns=['Adr', 'Status', 'Ort'])


In [ ]:
#überprüfen
df_cost.head()

In [ ]:
#Darstellung, sodass pro Jahr und Grossregion die Summe der Kosten aus ambulant/akut und stationär/akut 
#sicherstellen, dass die Spalten KostAmbA und KostStatA numerisch sind
cost_cols = ['KostAmbA', 'KostStatA']
df_cost[cost_cols] = df_cost[cost_cols].apply(pd.to_numeric, errors='coerce')

#aggregiert: Summe pro Grossregion pro Jahr
df_region_year = (
    df_cost
    .groupby(['JAHR', 'Grossregion'], as_index=False)[cost_cols]
    .sum()   # skipna=True standard
    .sort_values(['JAHR', 'Grossregion'])
)

#Schweiz als zusätzliche Kategorie anhängen
# Schweiz nur aus den Regionen bilden (vorhandene Schweiz-Zeilen ignorieren)
df_ch = (
    df_region_year[df_region_year['Grossregion'] != 'Schweiz']
    .groupby('JAHR', as_index=False)[cost_cols]
    .sum(min_count=1)
    .assign(Grossregion='Schweiz')
)
df_region_year = pd.concat([
    df_region_year[df_region_year['Grossregion'] != 'Schweiz'],
    df_ch
], ignore_index=True).sort_values(['JAHR', 'Grossregion'])



In [ ]:
check = (
    df_region_year[df_region_year['Grossregion'] != 'Schweiz']
    .groupby('JAHR')[cost_cols].sum()
    - df_region_year[df_region_year['Grossregion'] == 'Schweiz']
      .set_index('JAHR')[cost_cols]
)
check.abs().max()   # sollte 0 (oder nur Rundungsreste) sein

In [ ]:
df_region_year.head(8)

In [ ]:
df_region_year.query("Grossregion == 'Schweiz'")

In [ ]:
#Umbenennung/Standardisierung 
df_cost_ready = df_region_year.rename(columns={
    'JAHR': 'Jahr',
    'Grossregion': 'Grossregion',
    'KostAmbA': 'KostAmbA',
    'KostStatA': 'KostStatA'
})

#Datensatz Cost ist ready :)

In [ ]:
df_cost_ready.head(112)
df_cost_ready.iloc[50:100]

In [ ]:
#als nächstes kümmern wir uns um die weiteren drei Datensätze

df_staff = pd.read_excel('data/staff.xlsx', skiprows=2) #skipping the first two rows as they only contain the description / title
df_services = pd.read_excel('data/services.xlsx', skiprows=2)
df_infra = pd.read_excel('data/infrastructure.xlsx', skiprows=2)

In [ ]:
#having a look at the datasets to get a feeling how to merge them into one
df_staff.info()
df_staff.head()

In [ ]:
df_infrastructure.info()
df_infrastructure.head()

In [ ]:
df_services.info()
df_services.head()

In [ ]:
#before merging we're doing a rough clean so merging will be easier
#first: remove the first and eigth to eleventh column in the staff dataset
df_staff = df_staff.drop(columns=["Unnamed: 0", "Unnamed: 1", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10", "Unnamed: 11"]) #removing it this way to avoid removing more columns later on with another name if i run the code again (dont know if that makes sense)
df_staff.head()

In [ ]:
#step 2: (re)name the columns
#for df_staff
df_staff.columns = ["Berufsgruppe_ID",
                    "Berufsgruppe", 
                    "Krankenhaus_Typ_ID", 
                    "Krankenhaus_Typ", 
                    "Region_ID", 
                    "Region", 
                    "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
df_staff.head()

#for df_services
df_services.columns = ["Kat_Services",
                       "Kat_Services_Beschreibung",
                       "Region_ID", #nachher noch duplicates löschen! sind in jedem dataset vorhanden
                       "Region", #nachher noch duplicates löschen! sind in jedem dataset vorhanden
                       "Krankenhaus_Typ_ID", #nachher noch duplicates löschen! sind in jedem dataset vorhanden
                       "Krankenhaus_Typ", #nachher noch duplicates löschen! sind in jedem dataset vorhanden
                       "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
df_services.head()

#for df_infrastructure
df_infrastructure.columns = ["Gerät_Kurzname",
                             "Gerät_Beschreibung",
                             "Kat_Infrastructure", #Anzahl_Infra / Untersuchungen_Total
                             "Kat_Infrastructure_Beschreibung",
                             "Region_ID",
                             "Region",
                             "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023"]
df_infrastructure.head

In [ ]:
#step 3: bring the three datasets staff, services and infrastructure in a (not already perfect) tidy data layout before looking at the cost dataset
#generating "new" datasets to still have the "original version" if something goes wrong, maybe changing the name later on to make it more systematic
#for pd_staff
id_vars = ["Berufsgruppe_ID", "Berufsgruppe", "Krankenhaus_Typ_ID", "Krankenhaus_Typ", "Region_ID", "Region"] #define all variables that should stay in pd_staff
df_staff_melt = df_staff.melt(id_vars = id_vars, var_name = "Jahr", value_name = "Anzahl VZÄ") #melt the years in staff into a column

###recognized the data has a lot of NaN because e.g. the berufsgruppe_id is only for the first observation, so adding the first value for all the following ones
### so we have to add them with df.fillna (see cheat sheet)
columns_staff_ffill =["Berufsgruppe_ID", "Berufsgruppe", "Krankenhaus_Typ_ID", "Krankenhaus_Typ"] #creating a list which mentions for which columns we want fo ffill NaN with the values above, had an error when i ran the code without defining the columns
df_staff_melt[columns_staff_ffill] = df_staff_melt[columns_staff_ffill].ffill()
df_staff_melt.head(50)

In [ ]:
###for pd_services###
id_vars = ["Kat_Services", "Kat_Services_Beschreibung", "Region_ID", "Region", "Krankenhaus_Typ_ID", "Krankenhaus_Typ"] #define all variables that should stay in pd_services (can just overwrite the list above)
df_services_melt = df_services.melt(id_vars = id_vars, var_name = "Jahr", value_name = "Anzahl Kat_Services") #noch Kat-Services besser machen, so dass Anzahl Spitäler eine Spalte, Anzahl Betten etc.

#fill the missing values with df.fillna
columns_services_ffill = ["Kat_Services", "Kat_Services_Beschreibung", "Region_ID", "Region"]
df_services_melt[columns_services_ffill] = df_services_melt[columns_services_ffill].ffill()
df_services_melt.head(50)

### pivoting to get the Kat_Services column as seperate colums (sodass Anzahl_kat_services unterteilt wird und Anzahl Spitäler, Anzahl Betten etc. eigene spalten sind)
#drop the Kat_Services_Beschreibung to make it easier
df_services_melt = df_services_melt.drop(columns=["Kat_Services_Beschreibung"])
###pivoting
df_services_melt = df_services_melt.pivot(index=["Region_ID", "Region", "Krankenhaus_Typ_ID", "Krankenhaus_Typ", "Jahr"], columns="Kat_Services", values="Anzahl Kat_Services").reset_index()
df_services_melt.head(20)

In [ ]:
###for pd.infrastructure
id_vars = ["Gerät_Kurzname", "Gerät_Beschreibung", "Kat_Infrastructure", "Kat_Infrastructure_Beschreibung", "Region_ID", "Region"] #define all variables that should stay in pd_infrastructure
df_infrastructure_melt = df_infrastructure.melt(id_vars = id_vars, var_name = "Jahr", value_name = "Anzahl Infrastructure") #melt the years into a column

#using fffilna again
columns_infrastructure_ffill = ["Gerät_Kurzname", "Gerät_Beschreibung", "Kat_Infrastructure", "Kat_Infrastructure_Beschreibung"]
df_infrastructure_melt[columns_infrastructure_ffill] = df_infrastructure_melt[columns_infrastructure_ffill].ffill()
df_infrastructure_melt.head()


###pivoting because "Untersuchungen" and "Anzahl Geräte is in one column" --> Kat_Infrastructure and Kat_Infrastructure_Beschreibung are duplicates, so dropping Beschreibung first
df_infrastructure_melt = df_infrastructure_melt.drop(columns=["Kat_Infrastructure_Beschreibung"])
df_infrastructure_melt = df_infrastructure_melt.pivot(index=["Gerät_Kurzname", "Gerät_Beschreibung", "Region_ID", "Region", "Jahr"], columns="Kat_Infrastructure", values="Anzahl Infrastructure").reset_index()
df_infrastructure_melt.head(60)

df_infrastructure.drop(columns=['Gerät_Kurzname', ])

Getting an overview over the datasets and sorting them the same way to see what has to be done now

1) Tidying services

In [ ]:
###dropping the Gerät_Beschreibung, created a new cell above, initially i did this in the pivoting sector below but than i cant run the code again because it gives an error the column doesnst exist anymore
df_services_melt = df_services_melt.drop(columns=["Krankenhaus_Typ_ID"])

In [ ]:
###noticed in the code below (while pivoting) that we have some duplicate values why which we cant pivot, so we tried identifying them and dropping them
###df_services_melt.head(50)
###df_services_melt.duplicated(subset=["Region_ID", "Region", "Jahr", "Krankenhaus_Typ"]).sum()

###code isn't useful anymore as i just adjusted the data in the excel cause i wasnt able to do it in python


In [ ]:
df_services_melt = df_services_melt.sort_values(by=["Jahr", "Region_ID"])
df_services_melt.head()

###changing the different Krankenhaus_Typs to different columns
df_services_melt = df_services_melt.pivot(index=["Region_ID", "Region", "Jahr"], columns="Krankenhaus_Typ", values=["Anzahl_Spitaeler", "Betten_Total", "Betten_akut", "Betten_geb", "Betten_psy", "Betten_reha", "Bettenbelegung", "Bettenbetriebstage", "Pflegetage_M_365"]).reset_index()
df_services_melt.head()

In [ ]:
df_services_melt.head()

In [ ]:
###renaming the columns to get a better oversight and information
df_services_melt.columns = ["Region_ID", "Region", "Jahr", "Anzahl_Spitaeler_GrVe", "Anzahl_Spitaeler_ZeVe", "Anzahl_Spitaeler_AllgKr",
    "Betten_Total_GrVe", "Betten_Total_ZeVe", "Betten_Total_AllgKr", "Betten_akut_GrVe", "Betten_akut_ZeVe", "Betten_akut_AllgKr",
    "Betten_geb_GrVe", "Betten_geb_ZeVe", "Betten_geb_AllgKr", "Betten_psy_GrVe", "Betten_psy_ZeVe", "Betten_psy_AllgKr",
    "Betten_reha_GrVe", "Betten_reha_ZeVe", "Betten_reha_AllgKr", "Bettenbelegung_GrVe", "Bettenbelegung_ZeVe", "Bettenbelegung_AllgKr",
    "Bettenbetriebstage_GrVe", "Bettenbetriebstage_ZeVe", "Bettenbetriebstage_AllgKr", "Pflegetage_M_365_GrVe", "Pflegetage_M_365_ZeVe", "Pflegetage_M_365_AllgKr"]

df_services_melt.head()

2. Tidying df_infrastructure so every "Gerät" and "Untersuchungen_Gerät" is a single column

In [ ]:
###dropping the Gerät_Beschreibung, created a new cell above, initially i did this in the pivoting sector below but than i cant run the code again because it gives an error the column doesnst exist anymore
df_infrastructure_melt = df_infrastructure_melt.drop(columns=["Gerät_Beschreibung"])

In [ ]:
df_infrastructure_melt = df_infrastructure_melt.sort_values(by=["Jahr", "Region_ID"])
df_infrastructure_melt.head(10)


###changing Gerät_Untersuchungen and Anzahl_Gerät to a column so we have one line per year and region
df_infrastructure_melt = df_infrastructure_melt.pivot(index=["Region_ID", "Region", "Jahr"], columns="Gerät_Kurzname", values=["Anzahl_Infra", "Untersuchungen_Total"]).reset_index()
df_infrastructure_melt.head(20)

###renaming the columns
df_infrastructure_melt.columns = ["Region_ID", "Region", "Jahr", "ANGIOGRAPHIE_Geräte", "CT_SCANNER_Geräte", "DIALYSE_Geräte", "GAMMA_CAMERA_Geräte", "LINEARBESCHLEUNIGER_Geräte", "LITHOTRIPTOR_Geräte",
                                  "MRI_Geräte", "PET_SCANNER_Geräte", "ANGIOGRAPHIE_Untersuchungen", "CT_SCANNER_Untersuchungen", "DIALYSE_Untersuchungen", "GAMMA_CAMERA_Untersuchungen", "LINEARBESCHLEUNIGER_Untersuchungen",
                                  "LITHOTRIPTOR_Untersuchungen", "MRI_Untersuchungen", "PET_SCANNER_Untersuchungen"]
df_infrastructure_melt.head(20)


3. Tidying staff

In [ ]:
df_staff_melt = df_staff_melt.sort_values(by=["Jahr", "Region_ID"])
df_staff_melt.head()

In [ ]:
###dropping Berufsgruppe_ID
df_staff_melt = df_staff_melt.drop(columns=["Berufsgruppe_ID", "Krankenhaus_Typ_ID"])
df_staff_melt.head()

In [ ]:
###changing the different Berufsgruppen to different columns
df_staff_melt = df_staff_melt.pivot(index=["Krankenhaus_Typ", "Region_ID", "Region", "Jahr"], columns="Berufsgruppe", values="Anzahl VZÄ").reset_index()
df_staff_melt.head()

In [ ]:
###rename the columns
df_staff_melt.columns = ["Krankenhaus_Typ", "Region_ID", "Region", "Jahr", "MedTechPersonal_Anz", "MedTheraPersonal_Anz", "Pflegepersonal_Anz", "Ärzteschaft_Anz"]
df_staff_melt.head()

In [ ]:
###as for services, differ between the Krankenhaus_Typen
df_staff_melt = df_staff_melt.pivot(index=["Region_ID", "Region", "Jahr"], columns="Krankenhaus_Typ", values=["MedTechPersonal_Anz", "MedTheraPersonal_Anz", "Pflegepersonal_Anz", "Ärzteschaft_Anz"]).reset_index()
df_staff_melt.head()

In [ ]:
###rename the columns
df_staff_melt.columns = ["Region_ID", "Region", "Jahr", "MedTechPersonal_Anz_GrVe", "MedTechPersonal_Anz_ZeVe", "MedTechPersonal_Anz_AllgKr",
    "MedTheraPersonal_Anz_GrVe", "MedTheraPersonal_Anz_ZeVe", "MedTheraPersonal_Anz_AllgKr", "Pflegepersonal_Anz_GrVe", "Pflegepersonal_Anz_ZeVe", "Pflegepersonal_Anz_AllgKr",
    "Ärzteschaft_Anz_GrVe", "Ärzteschaft_Anz_ZeVe", "Ärzteschaft_Anz_AllgKr"]

df_staff_melt.head()

Getting again an overview of all datasets and seeing if theyre ready to merge

In [ ]:
df_services_melt.head()

In [ ]:
df_staff_melt.head()

In [ ]:
df_infrastructure_melt.head()

In [ ]:
df_region_year.head()